In [ ]:
# running this once for the first time.
!mkdir -p downloads intermediates outputs

# download character.txt from Linaqruf/animagine-xl
!wget -q --show-progress -c -P downloads/ "https://huggingface.co/spaces/Linaqruf/animagine-xl/resolve/main/wildcard/character.txt"
# download artist_dict from baqu2213/PoemForSmallFThings
!wget -q --show-progress -c -P downloads/ "https://huggingface.co/baqu2213/PoemForSmallFThings/raw/main/Danbooru%20Prompt%20Selector/TEST2024/artist_dictionary.py"
# download data.json from Linaqruf/1.2m-ordered-tags
!wget -q --show-progress -c -P downloads/ "https://huggingface.co/datasets/Linaqruf/1.2m-ordered-tags/resolve/main/data/data.json"
# make taglist.csv
!echo "making taglist.csv..."
!grep "ordered_tags" downloads/data.json | sed -e 's/^.*ordered_tags": "\(.*\)"$/\1/' > intermediates/taglist.csv

In [ ]:
import re
import sys
from downloads.artist_dictionary import artist_dict
import modifiers

# get character and series info from character.txt
file = "downloads/character.txt"
character_dict = {}
series_dict = {}
with open(file, 'r') as infile:
    for row in infile:
        _, character, series = row.replace('\\', '').split(', ')
        character_dict[character.strip().replace(' ', '_')] = 0
        series_dict[series.strip().replace(' ', '_')] = 0
infile.close()

character_list = list(character_dict.keys())
series_list = list(series_dict.keys())
artist_list = list(artist_dict.keys())


# read data from taglist and store to csv
file = "intermediates/taglist.csv"
unique_data = {}
with open(file, 'r') as infile:
    for row in infile:
        tags = row.strip().split(', ')
        category = 0
        after_rating = False;
        for tag in tags:
            tag = tag.replace('||| ', '').strip().replace(' ', '_')
            if not tag: # means empty
                continue

            if not tag in unique_data:
                if re.compile('(multiple_|[0-9]\+?)(boy|girl)s?$').match(tag) or 'male focus' in tag:
                    category = 0
                elif tag.replace('_', ' ') in artist_list:
                    category = 1
                elif tag in series_list:
                    category = 3
                elif tag in character_list:
                    category = 4
                elif tag.replace('_', ' ') in modifiers.year or tag.replace('_', ' ') in modifiers.quality or 'rating:' in tag:
                    category = 5 # not actually meta btw
                else:
                    category = 0 if after_rating else 3 # unknown character or series
                    
                unique_data[tag] = { "category": category, "count": 0 }
            unique_data[tag]["count"] = unique_data[tag]["count"] + 1

            if 'rating:' in tag:
                after_rating = True
infile.close()

ordered_data = dict(sorted(unique_data.items(), key=lambda x: x[1]["count"], reverse=True))
with open("outputs/animagine-xl-3.0.csv", 'w') as outfile:
    for key, value in ordered_data.items():
        outfile.write(f"{key},{value['category']},{value['count']},\n")
outfile.close()